In [ ]:
#Import Libraries
import pandas as pd
import json
import numpy as np
import re
import random


In [ ]:
#Format meter data to be compatible with Pandas
with open('data/parkingMeters.json') as f:
    data = json.load(f)

json_df_format = {}

for meter in data['features']:
  attributes = meter.pop('attributes')
  geometry = meter.pop('geometry')
  attributes['LATITUDE'] = geometry['y']
  attributes['LONGITUDE'] = geometry['x']
  meter_id = attributes.pop('METER_ID')
  json_df_format[meter_id] = attributes

In [ ]:
#Import formatted meter data
with open('data/formattedMeterData.json', 'w') as fp:
    json.dump(json_df_format, fp)
meters = pd.read_json('data/formattedMeterData.json', orient='index')

#Remove Taxi Zones
meters['TAXI_ZONE_HOURS'] = meters['TAXI_ZONE_HOURS'].apply(lambda x: str(x))
meters = meters[meters['TAXI_ZONE_HOURS'] == 'None']

#Remove Handicap Zones
meters = meters[meters['HANDICAP'] == 0]

#Text to Hour of Day
text_to_hod = {'3AM':3, '4PM':16, '3PM':15, '7AM':7, '6PM':18, '10AM':10, '9AM':9, '6AM':6, 'None':0, '0':0}
meters['TOW_AWAY_HOURS'] = meters['TOW_AWAY_HOURS'].apply(lambda x: str(x))
meters['TOW_AWAY_START'] = meters['TOW_AWAY_HOURS'].apply(lambda x: text_to_hod[x.split('-')[0].replace(" ","")])
meters['TOW_AWAY_END'] = meters['TOW_AWAY_HOURS'].apply(lambda x: text_to_hod[x.split('-')[-1].replace(" ","")])

#Only extract meter locations
meters = meters[['LATITUDE','LONGITUDE']]
meters.head()